# Data Analysis and Cleaning
analysis data and do some dirty work for cleaning
- <a href='#1'>1. eval the test set distribution</a> 
- <a href='#2'>2. task2</a> 
- <a href='#3'>3. task3</a>

In [9]:
! pip install chart-studio -i http://mirrors.aliyun.com/pypi/simple/ --trusted-host mirrors.aliyun.com
import sys
import os 

import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

sys.path.append('../')
import conf

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/


In [8]:
# global settings
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',100)
sns.set(rc={'figure.figsize':(11,4)})
cf.go_offline()

NameError: name 'cf' is not defined

In [3]:
# global variables
DEFAULT_MISSING_VALUE = 0
FONT = fm.FontProperties(fname = os.path.join(conf.LIB_DIR,'simsun.ttc'))

In [4]:
# functions
def __dummy():
    pass

def diivide_files_into_raw_and_normalized():
    pass

### <a id='1'> 1.eval the test set distribution</a>